In [3]:
#%load_ext autoreload
#%autoreload 2
from pgmpy.models import BayesianModel
from pgmpy.inference import VariableElimination
from pgmpy.readwrite import XMLBIFReader
from pgmpy.factors.discrete import TabularCPD

In [11]:
import gzip
import re
from collections import defaultdict

import numpy as np
import pandas as pd

In [5]:
review_full = pd.read_csv('data/review.tsv.gz',sep='\t')
tip_full = pd.read_csv('data/tip.tsv.gz',sep='\t')
business_full = pd.read_csv('data/business.tsv.gz',sep='\t')
checkin_full = pd.read_csv('data/checkin.tsv.gz',sep='\t')

/usr/local/lib/python2.7/dist-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (1,3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
#rs = review_full.drop(['cool','date','funny','type','useful','user_id'],1)
#rs = rs[['business_id','text','stars']]
#rs.to_csv('out.dat',header=False,sep='|')

In [12]:
if True:
    user_valid = review_full.groupby('user_id').count()
    review_sub = review_full[review_full.user_id.isin(list(user_valid[user_valid.business_id > 20].index))]

    print review_full.shape,review_sub.shape,business_full.shape,tip_full.shape,checkin_full.shape
    review = review_sub#.sample(frac=0.01, replace=False)
    business = business_full[business_full.business_id.isin(list(review.business_id.unique()))]
    tip = tip_full[tip_full.business_id.isin(list(review.business_id.unique()))]
    checkin = checkin_full[checkin_full.business_id.isin(list(review.business_id.unique()))]
    print review.shape,business.shape,tip.shape,checkin.shape
else:
    review = review_full
    tip = tip_full
    business = business_full
    checkin = checkin_full

(2577313, 10) (797099, 10) (48485, 16) (602833, 6) (46417, 3)
(797099, 10) (45352, 16) (599707, 6) (43848, 3)


In [13]:
categories = defaultdict(int)
for row,cnt in zip(business.categories,business.review_count):
    for ct in row.strip('[]').split(','):
        categories[ct.strip()] += cnt
cats = sorted([(v,k) for k,v in categories.iteritems()])[::-1]

In [14]:
good_cats = []

if False:
    small_bus = business.loc[:,['business_id','categories']]
    for cnt, cat in cats:
        small_bus[cat] = small_bus.categories.str.contains(re.escape(cat))
    small_bus = small_bus.loc[:,[k for k in small_bus.columns if k != 'categories']]
    users = review.loc[:,['user_id','business_id','stars']]
    all_reviews = users.merge(small_bus,on='business_id')
    log_or = all_reviews.groupby('user_id').sum()
    if True:
        with open("filtered_100.txt") as fp:
            for line in fp:
                cnt, label = line.split(',')
                good_cats.append(label.strip())

    cand_cats = [cat for cnt,cat in cats if cat not in good_cats]
    while len(cand_cats) > 0:
        cnts = []
        if len(good_cats) > 0:
            good_filter = log_or[good_cats[0]] > 0
            for i in xrange(1,len(good_cats)):
                good_filter = good_filter & (log_or[good_cats[1]] > 0)
        for cat in cand_cats:
            if len(good_cats) > 0:
                cnts.append(((log_or[(log_or[cat] >0) & good_filter].shape[0],cat)))
            else:
                cnts.append((log_or[log_or[cat] > 0].shape[0],cat))

        cnts = sorted(cnts)[::-1]
        if cnts[0][0] > 500:
            good_cats.append(cnts[0][1])
            cand_cats.remove(cnts[0][1])
            print cnts[0]
        else:
            break

In [15]:
if len(good_cats) > 0:
    valid_cats = good_cats[:]
    print len(valid_cats)
elif True:
    with open("filtered_100.txt") as fp:
        for line in fp:
            cnt, label = line.split(',')
            good_cats.append(label.strip())
    valid_cats = good_cats[:50]
    print len(valid_cats)
else:
    print len(cats)
    valid_cats = [k for v,k in cats[:35]]
    print valid_cats

small_bus = business.loc[:,['business_id','categories']]
small_bus['biz_review'] = business.stars
for cat in valid_cats:
    small_bus[cat] = small_bus.categories.str.contains(re.escape(cat))
small_bus = small_bus.loc[:,[k for k in small_bus.columns if k != 'categories']]

50


In [16]:
users = review.loc[:,['user_id','business_id','stars']]

In [17]:
#users['liked'] = users.stars > 3.0

In [18]:
all_reviews = users.merge(small_bus,on='business_id')

In [ ]:
#liking

grps = all_reviews.groupby(['business_id']).count() > 200
valid_biz = grps['stars'][grps['stars'] == True].index

sub_reviews = all_reviews[all_reviews.business_id.isin(valid_biz)]
#sub_reviews = all_reviews 

grps = sub_reviews.groupby(['user_id']).count() > 20
valid_users = grps['stars'][grps['stars'] == True].index
sub_reviews = sub_reviews[sub_reviews.user_id.isin(valid_users)]

new_reviews = pd.DataFrame()
new_groups = {
    'Asian': ['Asian Fusion','Chinese','Japanese','Sushi Bars','Thai','Vietnamese','Noodles','Korean'],
    'Morning': ['Breakfast & Brunch','Coffee & Tea','Cafes','Diners'],
    'Night': ['Beer','Wine Bars'],
    'Fast': ['Burgers','Pizza','Fast Food','Delis','Chicken Wings','Hot Dogs']
}
for cat in valid_cats:
    #new_reviews[cat]= all_reviews[all_reviews[cat]].groupby('user_id').mean()['stars'] > 3
    #new_reviews[cat]= all_reviews[all_reviews[cat]].groupby('user_id').mean()['stars'] > all_reviews[all_reviews[cat]].groupby('user_id').mean()['biz_review']
    #new_reviews[cat]= all_reviews[all_reviews[cat]].groupby(['user_id','business_id']).mean()['stars'] > all_reviews[all_reviews[cat]].groupby(['user_id','business_id']).mean()['biz_review']
    new_reviews[cat]= sub_reviews[sub_reviews[cat]].groupby('user_id').mean()['stars'] > sub_reviews[sub_reviews[cat]].groupby('user_id').mean()['biz_review']
data_sample = new_reviews

if True:
    for cat in new_groups:
        oc = new_groups[cat]
        g1 = sub_reviews[sub_reviews[oc[0]]].groupby('user_id').sum()
        for i in xrange(1,len(oc)):
            g1 += sub_reviews[sub_reviews[oc[i]]].groupby('user_id').sum()
        filered = g1#sub_reviews[sub_reviews[]].groupby('user_id').mean()
        data_sample[cat]= filered['stars'] > filered['biz_review']
data_sample.columns = [x.translate(None,' ()&/-') for x in data_sample.columns]
#visting
#from sklearn.linear_model import Lasso
#ds = data_sample.copy().transpose()
#for did, data in data_sample.transpose().iteritems():
#    clf = Lasso(0.001,fit_intercept=False)
#    clf.fit()

if True:
    import random
    MODEL_FILE = 'bayesMAN.bif.xml'
    reader = XMLBIFReader(MODEL_FILE)
    model = reader.get_model()
    belief_propagation = VariableElimination(model)
    invalid_cols = [x for x in data_sample.columns if x not in model.nodes()]
    try:
        data_sample = data_sample.drop(invalid_cols,1)
        data_sample = data_sample.drop(new_groups.keys(),1)
    except:
        pass
    print data_sample.shape
    correct = 0
    preds = 0
    for user,elem in data_sample.transpose().iteritems():
        vld = pd.DataFrame(elem.dropna() > 0).transpose()
        train = vld.sample(frac=0.8,axis=1)
        #print train.columns,vld.columns
        test = vld.drop(train.columns.tolist(),1)
        train_d = {k:int(v[0]) for k,v in train.iteritems()}
        test_d  = {k:int(v[0]) for k,v in test.iteritems()}
        for node in test_d:
            res = belief_propagation.query([node],evidence=train_d)[node]
            correct += int((res.values[1] > 0.5) == bool(test_d[node]))
            #correct += int((False) == bool(test_d[node]))
            preds += 1
    print float(correct)/float(preds)
#    print did, 
#data_sample = all_reviews.loc[:,[k for k in small_bus.columns if k not in ['user_id','business_id','stars']]]
data_sample.shape, data_sample.dropna().shape,all_reviews.shape # (1500, 21) or (28,51) or (500,35)

(520, 50)


In [167]:
#tmp_sample = data_sample#.dropna()
#int_cols = tmp_sample.fillna(0).astype(int)-1*((1.0 - tmp_sample.fillna(1).astype(int))) #astype(np.int)

#from pylab import *
#hist(np.array(int_cols.corr()).ravel())
#show()
#np.sum(int_cols.corr() > 0.25)
if False:
    data_sample.fillna(0)
    from sklearn.manifold import TSNE
    model = TSNE(n_components=2, init='random',metric='euclidean')
    pts = model.fit_transform(np.array(data_sample.fillna(0)).T)

In [168]:
if False:
    import matplotlib.pyplot as plt
    %matplotlib inline
    plt.style.use('ggplot')
    plt.figure(figsize=(10,10))
    plt.scatter(pts[:,0],pts[:,1])
    for i, txt in enumerate(data_sample.columns):
        plt.gca().annotate(txt, (pts[i,0],pts[i,1]))

# Model Fitting

In [21]:
from pgmpy.estimators import BdeuScore, K2Score, BicScore
from pgmpy.models import BayesianModel
from pgmpy.estimators import HillClimbSearch
from pgmpy.estimators import ConstraintBasedEstimator
from pgmpy.estimators import MaximumLikelihoodEstimator

In [22]:
#hc = HillClimbSearch(data_sample, scoring_method=BicScore(data_sample))
#best_model = hc.estimate()
#print(best_model.edges())

In [23]:
est = ConstraintBasedEstimator(data_sample,complete_samples_only=False)
data_sample.to_csv('bayes.csv')
data_sample.dropna().to_csv('bayes_small.csv')

In [32]:
if True:
    edges= [ ('BreakfastBrunch','CoffeeTea'),
             ('BreakfastBrunch','Cafes'),
             ('BreakfastBrunch','Diners'),
             ('Cafes','Bakeries'),
             ('Bakeries','Desserts'),
             ('Desserts','IceCreamFrozenYogurt'),
             ('AsianFusion','Chinese'),
             ('AsianFusion','SushiBars'),
             ('AsianFusion','Japanese'),
             ('SushiBars','Japanese'),
             ('AsianFusion','Thai'),
             ('AsianFusion','Vietnamese'),
             ('Japanese','Noodles'),
             ('AsianFusion','Korean'),
             ('Beer','WineBars'),
             ('FastFood','Burgers'),
             ('FastFood','Pizza'),
             ('FastFood','ChickenWings'),
             ('FastFood','Delis'),
             ('FastFood','HotDogs'),
             ('Diners','FastFood'),
             ('SushiBars','FastFood'),
             ('Delis','Sandwiches'),
             ('Sandwiches','Soup'),
             ('FastFood','Sandwiches'),
             ('Burgers','Sandwiches'),
             ('Burgers','Steakhouses'),
             ('AmericanTraditional','Steakhouses'),
             ('AmericanTraditional','FastFood'),
             ('AmericanTraditional','AmericanNew'),
             ('Steakhouses','Barbeque'),
             ('Southern','Barbeque'),
             ('Seafood','AmericanTraditional'),
             ('Southern','ComfortFood'),
             ('Steakhouses','Italian'),
             ('Italian','French'),
             ('Mexican','TexMex'),
             ('Mexican','LatinAmerican'),
             ('LatinAmerican','TapasBars'),
             ('TapasBars','TapasSmallPlates'),
             ('Vegetarian','Salad'),
             ('Vegetarian','Vegan'),
             ('JuiceBarsSmoothies','Vegetarian'),
             ('SpecialtyFood','EthnicFood'),
             ('SpecialtyFood','Mediterranean'),
             ('SpecialtyFood','MiddleEastern'),
             ('Mediterranean','Greek'),
             ('EthnicFood','Indian'),
             ('Salad','Buffets'),
             ('Indian','Buffets'),
             ('Vegetarian','Indian')]
    for e in edges:
        for i in xrange(2):
            if e[i] not in data_sample.columns:
                print e[i]
    nodes = valid_cats

else:
    clf = est.estimate(significance_level=0.01)
    print len(clf.edges())
    print clf.edges()
    edges = clf.edges()

In [33]:
model = BayesianModel(edges)
model.fit(data_sample,complete_samples_only=False)

In [34]:
for cpd in model.cpds:
    print cpd

+---------------------+----------------------------+---------------------------+
| AmericanTraditional | AmericanTraditional(False) | AmericanTraditional(True) |
+---------------------+----------------------------+---------------------------+
| AmericanNew(False)  | 0.703996726231             | 0.32601299103             |
+---------------------+----------------------------+---------------------------+
| AmericanNew(True)   | 0.296003273769             | 0.67398700897             |
+---------------------+----------------------------+---------------------------+
+----------------------------+----------------+----------------+
| Seafood                    | Seafood(False) | Seafood(True)  |
+----------------------------+----------------+----------------+
| AmericanTraditional(False) | 0.670975684826 | 0.383589059373 |
+----------------------------+----------------+----------------+
| AmericanTraditional(True)  | 0.329024315174 | 0.616410940627 |
+----------------------------+-------------

In [35]:

import pickle
with open('bayesMAN.pkl','wb') as fp:
    pickle.dump(model,fp)
from pgmpy.readwrite import *
writer = BIFWriter(model)
writer.write_bif('bayesMAN.bif')

writer = UAIWriter(model)
writer.write_uai('bayesMAN.uai')

writer = XMLBIFWriter(model)
writer.write_xmlbif('bayesMAN.bif.xml')

In [173]:
est = HillClimbSearch(data_sample.dropna(), scoring_method=BicScore(data_sample.dropna()))
best_model = est.estimate(max_indegree=2).edges()


KeyboardInterrupt: 